In [ ]:
pip install smolagents

In [ ]:
pip install litellm

In [ ]:
pip install groq

In [104]:
from smolagents import LiteLLMModel

model = LiteLLMModel(
    "gemini/gemini-2.0-flash",
    temperature=0.2,
    api_key="Gemini_api_key",
    max_token = 8000
)

In [32]:
from smolagents import Tool
import requests

class LinkedInProfileScraperTool(Tool):
    name = "linkedin_profile_scraper"
    description = "Scrapes LinkedIn profile data using the Fresh LinkedIn Profile Data API from RapidAPI."

    inputs = {
        "linkedin_url": {
            "type": "string",
            "description": "The LinkedIn profile URL to scrape (e.g., 'https://www.linkedin.com/in/username')."
        },
        "include_flags": {
            "type": "object",
            "description": "Optional dictionary of flags to include specific sections (e.g., {'include_skills': 'true'}).",
            "default": None,
            "nullable": True  # Allow None as a valid value
        }
    }

    output_type = "string"

    def __init__(self, api_key, **kwargs):
        super().__init__(**kwargs)
        self.api_url = "https://fresh-linkedin-profile-data.p.rapidapi.com/get-linkedin-profile-by-salesnavurl"
        self.headers = {
            "x-rapidapi-key": api_key,
            "x-rapidapi-host": "fresh-linkedin-profile-data.p.rapidapi.com"
        }



    def forward(self, linkedin_url: str, include_flags: dict = None):
        """
        Scrape LinkedIn profile data from the provided URL using the Fresh LinkedIn Profile Data API.

        Args:
            linkedin_url (str): The LinkedIn profile URL to scrape.
            include_flags (dict, optional): Flags to include specific sections (e.g., {'include_skills': 'true'}).

        Returns:
            str: The scraped profile data as a string or an error message.
        """
        # Construct the query parameters
        querystring = {"linkedin_url": linkedin_url}

        # Default all sections to "false" unless overridden by include_flags
        default_flags = {
            "include_skills": "false",
            "include_certifications": "false",
            "include_publications": "false",
            "include_honors": "false",
            "include_volunteers": "false",
            "include_projects": "false",
            "include_patents": "false",
            "include_courses": "false",
            "include_organizations": "false"
        }

        # Update with user-provided flags if any
        if include_flags:
            default_flags.update(include_flags)

        querystring.update(default_flags)

        try:
            # Send the GET request to the RapidAPI endpoint
            response = requests.get(self.api_url, headers=self.headers, params=querystring)
            response.raise_for_status()  # Raise an exception for bad status codes (e.g., 400, 401)

            # Return the response as a JSON string
            return str(response.json())

        except requests.RequestException as e:
            return f"Error scraping LinkedIn profile '{linkedin_url}': {str(e)}"
        except Exception as e:
            return f"Error processing request: {str(e)}"

In [55]:
from smolagents import Tool
import requests

class PDFCreatorTool(Tool):
    name = "pdf_creator"
    description = "Creates a PDF from HTML content using the YakPDF API from RapidAPI."

    inputs = {
        "html_content": {
            "type": "string",
            "description": "The HTML content to convert into a PDF."
        },
        "pdf_options": {
            "type": "object",
            "description": "Optional PDF formatting options (e.g., {'format': 'A4', 'scale': 1}).",
            "default": None,
            "nullable": True  # Allow None as a valid value
        }
    }

    output_type = "string"
    def __init__(self, api_key , **kwargs):
        super().__init__(**kwargs)
        self.api_url = "https://yakpdf.p.rapidapi.com/pdf"
        self.headers = {
            "x-rapidapi-key": api_key,
            "x-rapidapi-host": "yakpdf.p.rapidapi.com",
            "Content-Type": "application/json"
        }



    def forward(self, html_content: str, pdf_options: dict = None):
        """
        Create a PDF from the provided HTML content using the YakPDF API.

        Args:
            html_content (str): The HTML content to convert.
            pdf_options (dict, optional): PDF formatting options (e.g., {'format': 'A4'}).

        Returns:
            str: The PDF response as a string or an error message.
        """
        # Construct the payload
        payload = {
            "source": {"html": html_content},
            "pdf": {
                "format": "A4",
                "scale": 1,
                "printBackground": True
            },
            "wait": {
                "for": "navigation",
                "waitUntil": "load",
                "timeout": 2500
            }
        }

        # Update PDF options if provided
        if pdf_options:
            payload["pdf"].update(pdf_options)

        try:
            # Send the POST request to the YakPDF API
            response = requests.post(self.api_url, json=payload, headers=self.headers)
            response.raise_for_status()  # Raise an exception for bad status codes (e.g., 400, 401)
            pdf_data = response.content

            with open("/content/output.pdf", "wb") as f:
              f.write(pdf_data)

            print("PDF file saved as output.pdf")

            # Return the response content as a string (PDF binary data)
            return "PDF file saved as output.pdf "

        except requests.RequestException as e:
            return f"Error creating PDF: {str(e)}"
        except Exception as e:
            return f"Error processing request: {str(e)}"

In [102]:
from smolagents import Tool , CodeAgent
from groq import Groq

class GroqChatTool(Tool):
    name = "groq_chat"
    description = "Generates text completions using the Groq API based on a user prompt."

    inputs = {

        "html_content": {
            "type": "string",
            "description": "the content that have to convert to ui ",
            "default": None,
            "nullable": True  # Allow None as a valid value
        }
    }

    output_type = "string"
    def __init__(self, api_key , **kwargs):
        super().__init__(**kwargs)
        self.client = Groq(api_key=api_key)
        self.model = model




    def forward(self, html_content : str = "you are a helpful assistant"):
        """
        Generate a text completion using the Groq API.

        Args:
            html_content (str): The system message to guide the assistant's behavior.

        Returns:
            str: A highly complete html css ui fo given content
        """
        # Construct the messages list
        messages = [
            {"role": "system", "content": "ceate a good html css design from the text and only return code "},
            {"role": "user", "content": html_content }
        ]

        try:
            # Send the request to the Groq API
            chat_completion = self.client.chat.completions.create(
                messages=messages,
                model=self.model,
                max_completion_tokens=8092,
                top_p=1,  # Default from your example
                stop=None,  # Default from your example
                stream=False  # Default from your example
            )

            # Extract and return the completion
            return chat_completion.choices[0].message.content

        except Exception as e:
            return f"Error generating completion with Groq API: {str(e)}"

In [98]:
linked_scaper = LinkedInProfileScraperTool(api_key = "Rapid_api_key")

linked_resume = CodeAgent(
    tools=[linked_scaper],
    model=model,
    max_steps=10,
    additional_authorized_imports=['datetime', 'statistics', 'time', 'stat', 'queue', 're', 'unicodedata', 'collections', 'math',
'random', 'itertools'],
    name="linkedin_resume",
    description="creates a ell defined 2 page resume from the extracted data",
)

In [99]:
pdf_creation_tool = PDFCreatorTool(api_key = "Rapid_api_key")

pdf_creation = CodeAgent(
    tools=[pdf_creation_tool],
    model=model,
    max_steps=10,
    additional_authorized_imports=['datetime', 'statistics', 'time', 'stat', 'queue', 're', 'unicodedata', 'collections', 'math',
'random', 'itertools'],
    name="html_to_pdf",
    description="convert a html code to pdf and save the pdf ",
)

In [100]:
resume_to_ui = GroqChatTool(api_key = "Groq_api_key" )
resumetxt_to_html_css = CodeAgent(
    tools=[resume_to_ui],
    model=model,
    max_steps=10,
    additional_authorized_imports=['datetime', 'statistics', 'time', 'stat', 'queue', 're', 'unicodedata', 'collections', 'math',
'random', 'itertools'],
    name="resume_html_css",
    description="convert a string to highly stylish html css ui ",
)

In [101]:
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[linked_resume, pdf_creation, resumetxt_to_html_css],
    max_steps=10,
    additional_authorized_imports=['datetime', 'stat', 're', 'math', 'collections', 'itertools', 'statistics', 'time', 'queue',
'unicodedata', 'random'],

)

In [ ]:
manager_agent.run('create resume fo a developer from linkedin profile  https://www.linkedin.com/in/linkedin-profile-link-username/ then convert it to html and then to pdf ')